In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=a7f0d7c928127f341fc5eaa1e2b3f4aa0ed65dd8a9f51edf8c0af0c9aa00ffc9
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master('local[4]').appName('joins_hw').getOrCreate()

In [3]:
# Создаём датасет для примеров
dataset_1 = [
  {
    'key_1' : 'abc',
    'value_1' : 10,
    'value_2' : 20
  },
  {
    'key_1' : 'def',
    'value_1' : 100,
    'value_2' : 300
  }
]

dataset_2 = [
  {
    'key_2' : 'abc',
    'value_1' : 5.5,
    'value_2' : 2.2
  },
  {
    'key_2' : 'xyz',
    'value_1' : 10.1,
    'value_2' : 13.5
  }
]

df1 = spark.createDataFrame(dataset_1)
print('df1')
df1.show()

df2 = spark.createDataFrame(dataset_2)
print('df2')
df2.show()

df1
+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  abc|     10|     20|
|  def|    100|    300|
+-----+-------+-------+

df2
+-----+-------+-------+
|key_2|value_1|value_2|
+-----+-------+-------+
|  abc|    5.5|    2.2|
|  xyz|   10.1|   13.5|
+-----+-------+-------+



In [4]:
# Создайте джойн, чтобы получить следующую таблицу
# +---+-------+-------+
# |key|value_1|value_2|
# +---+-------+-------+
# |abc|     10|     20|
# +---+-------+-------+

In [5]:
###Left Semi Join возвращает все записи из левого датафрейма, если соответствующее значение существует в правом датафрейме
###В соединении LEFT SEMI мы не увидим больше строк, чем ожидалось, даже если существует сопоставление один ко многим. Это похоже на состояние «IN».
###Выходной фрейм данных будет иметь столбцы только из левого датафрейма.

df1.join(df2, df1.key_1 == df2.key_2, 'left_semi').show()

+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  abc|     10|     20|
+-----+-------+-------+



In [6]:
# Создайте джойн, чтобы получить следующую таблицу
# +---+-------+-------+
# |key|value_1|value_2|
# +---+-------+-------+
# |def|    100|    300|
# +---+-------+-------+

In [7]:
###LEFT Anti Join возвращает все записи из левого датафрейма, которых нет в правом датафрейме. 
###Это противоположно соединению LEFT SEMI, которое возвращает только совпадающие записи. Это похоже на состояние «NOT IN». 
###Вывод будет иметь вывод только из левого датафрейма.

df1.join(df2, df1.key_1 == df2.key_2, 'left_anti').show()

+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  def|    100|    300|
+-----+-------+-------+



In [8]:
# Создайте Inner джойн с условиями ---hidden---, для df1 и df2, соответсвенно
# В  итоге получится таблица
# +---+-------+-------+---+-------+-------+
# |key|value_1|value_2|key|value_1|value_2|
# +---+-------+-------+---+-------+-------+
# |abc|     10|     20|abc|    5.5|    2.2|
# |abc|     10|     20|xyz|   10.1|   13.5|
# |def|    100|    300|abc|    5.5|    2.2|
# |def|    100|    300|xyz|   10.1|   13.5|
# +---+-------+-------+---+-------+-------+

In [9]:
###Inner join является соединением по умолчанию и выбирает строки, имеющие совпадающие (либо несовпадающие в зависимости от условия) значения в обоих отношениях.

df1.join(df2, df1.value_1 != df2.value_2).show()

+-----+-------+-------+-----+-------+-------+
|key_1|value_1|value_2|key_2|value_1|value_2|
+-----+-------+-------+-----+-------+-------+
|  abc|     10|     20|  abc|    5.5|    2.2|
|  abc|     10|     20|  xyz|   10.1|   13.5|
|  def|    100|    300|  abc|    5.5|    2.2|
|  def|    100|    300|  xyz|   10.1|   13.5|
+-----+-------+-------+-----+-------+-------+

